In [1]:
#| default_exp sandbox

In [2]:
from jupyterlab.extensions.pypi import https_proxy_url
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from dotenv import dotenv_values
config = dotenv_values(".env")
key = config.get('GC_QUOTE_API_CREDENTIALS')

In [6]:
from google.cloud import bigquery
import pandas_gbq
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file("../res/quote-api-365206-39d39b2fde07.json")

In [7]:
client = bigquery.Client(credentials=credentials)

QUERY = """
		select *
		from `quote-api-365206.report.ride_dispatch_v` r
		where dispatch_amount_net_usd is not null
        limit 200
		"""

In [8]:
QUERY_DISINCT = """
		select count (*)
		from `quote-api-365206.report.ride_dispatch_v` r
		where trip_type is null
        limit 200
		"""

In [9]:
!echo $HTTP_PROXY

http://127.0.0.1:20171


In [10]:
#query_job = client.query(QUERY)

In [11]:

df = pandas_gbq.read_gbq(QUERY,credentials=credentials)

Downloading: 100%|██████████|


In [12]:

df_trip_type = pandas_gbq.read_gbq(QUERY_DISINCT,credentials=credentials)

Downloading: 100%|██████████|


In [13]:

df.head()

,ride_id,trip_no,dispatch_id,trip_type,ride_stat,dispatch_stat,partner_tran_id,partner_id,from_fleet_id_as_customer,pickup_date,...,has_complaint,has_fleet_complaint,has_penalty_complaint,has_ops_complaint,gps_ad_check,gps_cns_check,score,incoming_dispatch_info_last_updated_at,dispatch_info_last_updated_at,etl_time
0,2180308,1,30975,None,Cancelled,At destination,16772,1,16,2021-07-11,...,0,0,0,0,0,0,None,2021-07-08 02:48:47+00:00,2021-07-11 22:17:56+00:00,2024-08-28 23:05:44.634866+00:00
1,2392467,0,505428,None,Accepted,At destination,230866,1,16,2022-11-12,...,1,0,1,1,0,0,None,2022-11-28 13:56:20+00:00,2022-11-28 13:56:41+00:00,2024-08-28 23:05:44.634866+00:00
2,2165291,0,1057,None,Cancelled,At destination,602,1,16,2020-07-11,...,0,0,0,0,0,0,None,2020-07-09 23:21:31+00:00,2020-07-11 00:38:55+00:00,2024-08-28 23:05:44.634866+00:00
3,2165060,0,601,None,Cancelled,At destination,324,1,16,2020-06-24,...,0,0,0,0,0,0,None,2020-06-22 05:12:23+00:00,2020-06-24 22:55:30+00:00,2024-08-28 23:05:44.634866+00:00
4,3531231,0,2800358,point2point,Cancelled,Accepted,1377216,1,16,2024-06-27,...,0,0,0,0,0,0,None,2024-06-13 13:12:21+00:00,2024-06-13 14:10:19+00:00,2024-08-28 23:05:44.634866+00:00


In [14]:
QUERY_ROUTE = """
        SELECT r.*, p1.lat from_lat, p1.lng from_lng, p2.lat to_lat, p2.lng to_lng
        FROM `report.ride_dispatch_v` r
        left JOIN `report.dim_place` p1
        ON r.pickup_place_id=p1.id
        INNER JOIN `report.dim_place` p2
        ON r.pickup_place_id=p2.id
        ORDER BY ride_id DESC
        limit 2;
		"""


In [15]:
df_route = pandas_gbq.read_gbq(QUERY_ROUTE,credentials=credentials)

Downloading: 100%|██████████|


In [16]:
df_route.head()

,ride_id,trip_no,dispatch_id,trip_type,ride_stat,dispatch_stat,partner_tran_id,partner_id,from_fleet_id_as_customer,pickup_date,...,gps_ad_check,gps_cns_check,score,incoming_dispatch_info_last_updated_at,dispatch_info_last_updated_at,etl_time,from_lat,from_lng,to_lat,to_lng
0,3763046,0,3212553,point2point,Pending,None,1581152,2692,57395,2024-11-11,...,0,0,None,2024-08-28 22:50:59+00:00,NaT,2024-08-28 23:05:44.634866+00:00,1.316602200,103.900220000,1.316602200,103.900220000
1,3763045,0,3212552,point2point,Pending,None,1581151,2,18,2024-10-19,...,0,0,None,2024-08-28 22:50:24+00:00,NaT,2024-08-28 23:05:44.634866+00:00,-33.410093000,-70.792723000,-33.410093000,-70.792723000


In [17]:
df_route.dtypes

ride_id                                                 Int64
trip_no                                                 Int64
dispatch_id                                             Int64
trip_type                                              object
ride_stat                                              object
dispatch_stat                                          object
partner_tran_id                                         Int64
partner_id                                              Int64
from_fleet_id_as_customer                               Int64
pickup_date                                            dbdate
pickup_time                                            dbtime
pickup_datetime                           datetime64[us, UTC]
pickup_local_datetime                          datetime64[us]
pickup_local_datetime_str                              object
pickup_place_id                                         Int64
dropoff_place_id                                        Int64
passenge